How to import these things?


In [1]:
# imports
from requests import get
from bs4 import BeautifulSoup
import os


In [40]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import pickle
import wrangle as w
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from collections import Counter
import random
import joblib
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import sys
import unicodedata
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /Users/click/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# need to pull list of top 100 top star usernames
# example: 'https://github.com/underdash/underdash/blob/master/README.md'
# so far from looking, its a 'span class=username/projectname'
# To break it down
# url = 'https://github.com/' + 'username/projectname' + '/blob/master/README.md'


In [2]:
"""
A module for obtaining repo readme and language data from the github API.
Before using this module, read through it, and follow the instructions marked
TODO.
After doing so, run it like this:
    python acquire.py
To create the `data.json` file that contains the data.
"""
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests

from env import github_token, github_username

# TODO: Make a github personal access token.
#     1. Go here and generate a personal access token https://github.com/settings/tokens
#        You do _not_ need select any scopes, i.e. leave all the checkboxes unchecked
#     2. Save it in your env.py file under the variable `github_token`
# TODO: Add your github username to your env.py file under the variable `github_username`
# TODO: Add more repositories to the `REPOS` list below.

REPOS = []
 

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        return repo_info.get("language", None)
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_contents = requests.get(get_readme_download_url(contents)).text
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    return [process_repo(repo) for repo in REPOS]


if __name__ == "__main__":
    data = scrape_github_data()
    json.dump(data, open("data2.json", "w"), indent=1)

In [4]:
url = 'https://github.com/search?q=stars%3A%3E0&s=stars&type=Repositories'

In [5]:
response_data = github_api_request(url)

In [6]:
response_data

{'payload': {'header_redesign_enabled': False,
  'results': [{'id': '21737465',
    'archived': False,
    'color': None,
    'followers': 273550,
    'has_funding_file': True,
    'hl_name': 'sindresorhus/awesome',
    'hl_trunc_description': '😎 Awesome lists about all kinds of interesting topics',
    'language': None,
    'mirror': False,
    'owned_by_organization': False,
    'public': True,
    'repo': {'repository': {'id': 21737465,
      'name': 'awesome',
      'owner_id': 170270,
      'owner_login': 'sindresorhus',
      'updated_at': '2023-10-17T13:44:41.058Z',
      'has_issues': True}},
    'sponsorable': True,
    'topics': ['lists', 'awesome', 'unicorns', 'resources', 'awesome-list'],
    'type': 'Public',
    'help_wanted_issues_count': 2,
    'good_first_issue_issues_count': 0,
    'starred_by_current_user': False},
   {'id': '177736533',
    'archived': False,
    'color': None,
    'followers': 268044,
    'has_funding_file': False,
    'hl_name': '996icu/996.ICU',


In [11]:
processed_repos = set()

In [12]:
repo_links = []
for result in response_data["payload"]["results"]:
    owner_login = result["repo"]["repository"]["owner_login"]
    repo_name = result["repo"]["repository"]["name"]
    repo_url = f"/{owner_login}/{repo_name}"
    if repo_url not in processed_repos:
        repo_links.append(repo_url)
        processed_repos.add(repo_url)

In [13]:
repo_links

['/sindresorhus/awesome',
 '/996icu/996.ICU',
 '/codecrafters-io/build-your-own-x',
 '/Significant-Gravitas/AutoGPT',
 '/jackfrued/Python-100-Days',
 '/avelino/awesome-go',
 '/justjavac/free-programming-books-zh_CN',
 '/kubernetes/kubernetes',
 '/puppeteer/puppeteer',
 '/iluwatar/java-design-patterns']

In [14]:
processed_repos

{'/996icu/996.ICU',
 '/Significant-Gravitas/AutoGPT',
 '/avelino/awesome-go',
 '/codecrafters-io/build-your-own-x',
 '/iluwatar/java-design-patterns',
 '/jackfrued/Python-100-Days',
 '/justjavac/free-programming-books-zh_CN',
 '/kubernetes/kubernetes',
 '/puppeteer/puppeteer',
 '/sindresorhus/awesome'}

Code from Zac for acquire

In [22]:
def fetch_github_data(url):
    """
    Fetch data from a GitHub API endpoint.

    Parameters:
    - url (str): The URL of the API endpoint.

    Returns:
    - dict: The JSON response from the API.
    """

    # Define the base URL for the GitHub search API
    BASE_URL = "https://github.com/search?q=stars%3A%3E0+language%3A{language}&type=repositories&l={language}&p={page}"

    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    elif response.status_code == 429:
        print(
            f"Status code: {response.status_code} - Rate limit exceeded. Ending script."
        )
        sys.exit()
    else:
        print(f"Status code: {response.status_code} - Failed to fetch data from {url}.")
        return None


In [27]:
BASE_URL = "https://github.com/search?q=stars%3A%3E0+language%3A{language}&type=repositories&l={language}&p={page}"

In [23]:


def extract_repo_links(data, processed_repos):
    """
    Extract repository links from the JSON response of a GitHub API endpoint.

    Parameters:
    - data (dict): The JSON response from the API.
    - processed_repos (set): A set of repository URLs that have already been processed.

    Returns:
    - list: A list of repository URLs that have not been processed before.
    """
    repo_links = []
    for result in data["payload"]["results"]:
        owner_login = result["repo"]["repository"]["owner_login"]
        repo_name = result["repo"]["repository"]["name"]
        repo_url = f"/{owner_login}/{repo_name}"
        if repo_url not in processed_repos:
            repo_links.append(repo_url)
            processed_repos.add(repo_url)
    return repo_links



In [24]:

def get_readme_content(repo):
    """
    Get the content of the README file for a repository.

    Parameters:
    - repo (str): The URL of the repository.

    Returns:
    - str: The content of the README file, or None if the file could not be found.
    """
    readme_url = "https://github.com" + repo + "/blob/master/README.md"
    response = requests.get(readme_url)
    soup = BeautifulSoup(response.content, "html.parser")
    readme_content = soup.select_one("article")
    return readme_content.get_text() if readme_content else None


In [25]:


def fetch_readmes(
    languages=["python"], num_repos=5, start_page=1, sleep_time=1, verbose=True
):
    """
    Fetch README files from GitHub repositories.

    Parameters:
    - languages (list): A list of programming languages to search for.
    - num_repos (int): The number of repositories to fetch README files from.
    - start_page (int): The page number to start searching from.
    - sleep_time (int): The number of seconds to wait between requests.
    - verbose (bool): Whether to print progress messages.

    Returns:
    - pandas.DataFrame: A DataFrame containing the language, repository URL, and README content for each fetched README file.
    """
    all_readmes = []
    processed_repos = set()

    for language in languages:
        if verbose:
            print(f"Processing {language} repositories...")
        page = start_page
        total_processed = 0

        while total_processed < num_repos:
            url = BASE_URL.format(language=language, page=page)
            data = fetch_github_data(url)
            if not data:
                print(f"Failed to fetch data for {language} on page {page}.")
                break

            repo_links = extract_repo_links(data, processed_repos)
            for repo in repo_links:
                if total_processed >= num_repos:
                    break
                readme_content = get_readme_content(repo)
                if readme_content:
                    all_readmes.append((language, repo, readme_content))
                    total_processed += 1
                    if verbose:
                        print(
                            f"Fetched README {total_processed} of {num_repos} for {language}."
                        )
                time.sleep(sleep_time)  # Add a delay between requests
            page += 1

        if num_repos > 10 and page <= num_repos // 10:
            if verbose:
                print(f"Fetching additional pages for {language}...")
            for i in range(page, num_repos // 10 + 1):
                url = BASE_URL.format(language=language, page=i)
                data = fetch_github_data(url)
                if not data:
                    print(f"Failed to fetch data for {language} on page {i}.")
                    break

                repo_links = extract_repo_links(data, processed_repos)
                for repo in repo_links:
                    if total_processed >= num_repos:
                        break
                    readme_content = get_readme_content(repo)
                    if readme_content:
                        all_readmes.append((language, repo, readme_content))
                        total_processed += 1
                        if verbose:
                            print(
                                f"Fetched README {total_processed} of {num_repos} for {language}."
                            )
                    time.sleep(sleep_time)  # Add a delay between requests

            if verbose:
                print(
                    f"Finished processing {language} repositories. Fetched a total of {total_processed} READMEs."
                )

    df = pd.DataFrame(all_readmes, columns=["language", "repo", "readme"])
    if verbose:
        print(f"Total README Count: {len(df)}")
    return df

In [20]:
fetch_github_data(url)

{'payload': {'header_redesign_enabled': False,
  'results': [{'id': '21486287',
    'archived': False,
    'color': '#3178c6',
    'followers': 7885,
    'has_funding_file': False,
    'hl_name': 'sahat/satellizer',
    'hl_trunc_description': 'Token-based AngularJS Authentication',
    'language': 'TypeScript',
    'mirror': False,
    'owned_by_organization': False,
    'public': True,
    'repo': {'repository': {'id': 21486287,
      'name': 'satellizer',
      'owner_id': 544954,
      'owner_login': 'sahat',
      'updated_at': '2022-07-06T20:42:44.738Z',
      'has_issues': True}},
    'sponsorable': False,
    'topics': [],
    'type': 'Public',
    'help_wanted_issues_count': 0,
    'good_first_issue_issues_count': 0,
    'starred_by_current_user': False},
   {'id': '9950667',
    'archived': True,
    'color': '#00ADD8',
    'followers': 6639,
    'has_funding_file': False,
    'hl_name': 'go-xorm/xorm',
    'hl_trunc_description': 'Simple and Powerful ORM for Go, support mysq

In [28]:
python = fetch_readmes(languages=['python'], num_repos=100, verbose=True, sleep_time=5)

Processing python repositories...
Fetched README 1 of 100 for python.
Fetched README 2 of 100 for python.
Fetched README 3 of 100 for python.
Fetched README 4 of 100 for python.
Fetched README 5 of 100 for python.
Fetched README 6 of 100 for python.
Fetched README 7 of 100 for python.
Fetched README 8 of 100 for python.
Fetched README 9 of 100 for python.
Fetched README 10 of 100 for python.
Fetched README 11 of 100 for python.
Fetched README 12 of 100 for python.
Fetched README 13 of 100 for python.
Fetched README 14 of 100 for python.
Fetched README 15 of 100 for python.
Fetched README 16 of 100 for python.
Fetched README 17 of 100 for python.
Fetched README 18 of 100 for python.
Fetched README 19 of 100 for python.
Fetched README 20 of 100 for python.
Fetched README 21 of 100 for python.
Fetched README 22 of 100 for python.
Fetched README 23 of 100 for python.
Fetched README 24 of 100 for python.
Fetched README 25 of 100 for python.
Fetched README 26 of 100 for python.
Fetched READM

In [29]:
python.to_csv('python_readmes.csv')

In [30]:
python_df = pd.read_csv('python_readmes.csv')

In [31]:
python_df.head()

,Unnamed: 0,language,repo,readme
0,0,python,/ethereon/caffe-tensorflow,Caffe to TensorFlow\nConvert Caffe models to T...
1,1,python,/nanohop/sketch-to-react-native,Convert Sketch files to React Native component...
2,2,python,/jeffknupp/sandman,sandman\n\n\n\n\n\nHomepage\nVisit the home of...
3,3,python,/CreatCodeBuild/TensorFlow-and-DeepLearning-Tu...,TensorFlow and DeepLearning Tutorial\nThis is ...
4,4,python,/wrobstory/vincent,Status\n2016-06-18 Update\nIf you are interest...


In [32]:
html = fetch_readmes(languages=['html'], num_repos=100, verbose=True, sleep_time=5)
html.to_csv('html_readmes.csv')
html_df = pd.read_csv('html_readmes.csv')

Processing html repositories...
Fetched README 1 of 100 for html.
Fetched README 2 of 100 for html.
Fetched README 3 of 100 for html.
Fetched README 4 of 100 for html.
Fetched README 5 of 100 for html.
Fetched README 6 of 100 for html.
Fetched README 7 of 100 for html.
Fetched README 8 of 100 for html.
Fetched README 9 of 100 for html.
Fetched README 10 of 100 for html.
Fetched README 11 of 100 for html.
Fetched README 12 of 100 for html.
Fetched README 13 of 100 for html.
Fetched README 14 of 100 for html.
Fetched README 15 of 100 for html.
Fetched README 16 of 100 for html.
Fetched README 17 of 100 for html.
Fetched README 18 of 100 for html.
Fetched README 19 of 100 for html.
Fetched README 20 of 100 for html.
Fetched README 21 of 100 for html.
Fetched README 22 of 100 for html.
Fetched README 23 of 100 for html.
Fetched README 24 of 100 for html.
Fetched README 25 of 100 for html.
Fetched README 26 of 100 for html.
Fetched README 27 of 100 for html.
Fetched README 28 of 100 for htm

In [33]:
html_df.head()

,Unnamed: 0,language,repo,readme
0,0,html,/tc39/proposal-optional-chaining,Optional Chaining for JavaScript\nStatus\nECMA...
1,1,html,/hackers-painters/samurai-native,\nBring web standards to native platform\nsamu...
2,2,html,/mritd/dockerfile,Dockerfile\n⚠️⚠️⚠️ Note: Some build work of do...
3,3,html,/superherojs/superherojs,Superhero.js\nMaintaining a large JavaScript c...
4,4,html,/quantifiedcode/python-anti-patterns,The Little Book of Python Anti-Patterns\nThis ...


In [34]:
javascript = fetch_readmes(languages=['javascript'], num_repos=100, verbose=True, sleep_time=5)
javascript.to_csv('javascript_readmes.csv')
javascript_df = pd.read_csv('javascript_readmes.csv')

Processing javascript repositories...
Fetched README 1 of 100 for javascript.
Fetched README 2 of 100 for javascript.
Fetched README 3 of 100 for javascript.
Fetched README 4 of 100 for javascript.
Fetched README 5 of 100 for javascript.
Fetched README 6 of 100 for javascript.
Fetched README 7 of 100 for javascript.
Fetched README 8 of 100 for javascript.
Fetched README 9 of 100 for javascript.
Fetched README 10 of 100 for javascript.
Fetched README 11 of 100 for javascript.
Fetched README 12 of 100 for javascript.
Fetched README 13 of 100 for javascript.
Fetched README 14 of 100 for javascript.
Fetched README 15 of 100 for javascript.
Fetched README 16 of 100 for javascript.
Fetched README 17 of 100 for javascript.
Fetched README 18 of 100 for javascript.
Fetched README 19 of 100 for javascript.
Fetched README 20 of 100 for javascript.
Fetched README 21 of 100 for javascript.
Fetched README 22 of 100 for javascript.
Fetched README 23 of 100 for javascript.
Fetched README 24 of 100 for

In [35]:
java = fetch_readmes(languages=['java'], num_repos=100, verbose=True, sleep_time=5)
java.to_csv('java_readmes.csv')
java_df = pd.read_csv('java_readmes.csv')

Processing java repositories...
Fetched README 1 of 100 for java.
Fetched README 2 of 100 for java.
Fetched README 3 of 100 for java.
Fetched README 4 of 100 for java.
Fetched README 5 of 100 for java.
Fetched README 6 of 100 for java.
Fetched README 7 of 100 for java.
Fetched README 8 of 100 for java.
Fetched README 9 of 100 for java.
Fetched README 10 of 100 for java.
Fetched README 11 of 100 for java.
Fetched README 12 of 100 for java.
Fetched README 13 of 100 for java.
Fetched README 14 of 100 for java.
Fetched README 15 of 100 for java.
Fetched README 16 of 100 for java.
Fetched README 17 of 100 for java.
Fetched README 18 of 100 for java.
Fetched README 19 of 100 for java.
Fetched README 20 of 100 for java.
Fetched README 21 of 100 for java.
Fetched README 22 of 100 for java.
Fetched README 23 of 100 for java.
Fetched README 24 of 100 for java.
Fetched README 25 of 100 for java.
Fetched README 26 of 100 for java.
Fetched README 27 of 100 for java.
Fetched README 28 of 100 for jav

In [36]:
df = pd.concat([java_df, python_df, javascript_df, html_df])

In [37]:
df.shape

(400, 4)

In [38]:
df.to_csv('all_readmes.csv')

In [39]:
df.head()

,Unnamed: 0,language,repo,readme
0,0,java,/roboguice/roboguice,"As of August 2016, RoboGuice is no longer supp..."
1,1,java,/kikoso/android-stackblur,Android StackBlur\nAndroid StackBlur is a libr...
2,2,java,/avast/android-butterknife-zelezny,ButterKnifeZelezny\nSimple plug-in for Android...
3,3,java,/bauerca/drag-sort-listview,DragSortListView\nNOTICE: No longer maintained...
4,4,java,/airbnb/native-navigation,Native Navigation\n\n\n\n\nDISCLAIMER\nThis pr...


In [196]:
df = pd.read_csv('all_readmes.csv')

In [197]:
df.head()

,Unnamed: 0.1,Unnamed: 0,language,repo,readme
0,0,0,java,/roboguice/roboguice,"As of August 2016, RoboGuice is no longer supp..."
1,1,1,java,/kikoso/android-stackblur,Android StackBlur\nAndroid StackBlur is a libr...
2,2,2,java,/avast/android-butterknife-zelezny,ButterKnifeZelezny\nSimple plug-in for Android...
3,3,3,java,/bauerca/drag-sort-listview,DragSortListView\nNOTICE: No longer maintained...
4,4,4,java,/airbnb/native-navigation,Native Navigation\n\n\n\n\nDISCLAIMER\nThis pr...


In [198]:
df.columns = df.columns.str.lower()

In [199]:
df.columns

Index(['unnamed: 0.1', 'unnamed: 0', 'language', 'repo', 'readme'], dtype='object')

In [200]:
df.columns = df.columns.str.replace(' ', '').str.replace(':', '')

In [201]:
df.drop(columns =['unnamed0.1', 'unnamed0'], inplace=True)

In [202]:
df.head()

,language,repo,readme
0,java,/roboguice/roboguice,"As of August 2016, RoboGuice is no longer supp..."
1,java,/kikoso/android-stackblur,Android StackBlur\nAndroid StackBlur is a libr...
2,java,/avast/android-butterknife-zelezny,ButterKnifeZelezny\nSimple plug-in for Android...
3,java,/bauerca/drag-sort-listview,DragSortListView\nNOTICE: No longer maintained...
4,java,/airbnb/native-navigation,Native Navigation\n\n\n\n\nDISCLAIMER\nThis pr...


## Prepare

In [203]:
def basic_clean(string):
    string = string.lower().replace('\\n', ' ')
    string = unicodedata.normalize('NFKD', string).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    string = re.sub(r"[^a-z0-9'\s]", '', string)

    return string


In [204]:
def tokenize(string):
    '''
    This function takes in a string
    and returns the string as individual tokens put back into the string
    '''
    #create the tokenizer
    tokenizer = nltk.tokenize.ToktokTokenizer()

    #use the tokenizer
    string = tokenizer.tokenize(string, return_str = True)

    return string

In [205]:
def stem(string):
    '''
    This function takes in text
    and returns the stem word joined back into the text
    '''
    #create porter stemmer
    ps = nltk.porter.PorterStemmer()
    
    #use the stem, split string using each word
    stems = [ps.stem(word) for word in string.split()]
    
    #join stem word to string
    string = ' '.join(stems)

    return string

In [206]:
def lemmatize(string):
    '''
    This function takes in a string
    and returns the lemmatized word joined back into the string
    '''
    #create the lemmatizer
    wnl = nltk.stem.WordNetLemmatizer()
    
    #look at the article 
    lemmas = [wnl.lemmatize(word) for word in string.split()]
    
    #join lemmatized words into article
    string = ' '.join(lemmas)

    return string

In [207]:
def remove_stopwords(string, exclude_words = []):
    '''
    This function takes in text, extra words and exclude words
    and returns a list of text with stopword removed
    '''
    #create stopword list
    stopword_list = stopwords.words('english')
    
    #remove excluded words from list
    stopword_list = set(stopword_list) - set(exclude_words)
    
    extra_words = ['javascript', 'java', 'python', 'html']
    #add the extra words to the list
    stopword_list = stopword_list.union(set(extra_words))
    
    #split the string into different words
    words = string.split()
    
    #create a list of words that are not in the list
    filtered_words = [word for word in words if word not in stopword_list]
    
    #join the words that are not stopwords (filtered words) back into the string
    string = ' '.join(filtered_words)
    
    return string

In [208]:
df['readme_clean'] = df.readme.apply(basic_clean).apply(tokenize).apply(remove_stopwords)

In [209]:
df.head()

,language,repo,readme,readme_clean
0,java,/roboguice/roboguice,"As of August 2016, RoboGuice is no longer supp...",august 2016 roboguice longer supported nearly ...
1,java,/kikoso/android-stackblur,Android StackBlur\nAndroid StackBlur is a libr...,android stackblur android stackblur library pe...
2,java,/avast/android-butterknife-zelezny,ButterKnifeZelezny\nSimple plug-in for Android...,butterknifezelezny simple plugin android studi...
3,java,/bauerca/drag-sort-listview,DragSortListView\nNOTICE: No longer maintained...,dragsortlistview notice longer maintained much...
4,java,/airbnb/native-navigation,Native Navigation\n\n\n\n\nDISCLAIMER\nThis pr...,native navigation disclaimer project currently...


In [210]:
df['readme_stemmed'] = df.readme_clean.apply(stem)

In [211]:
df.head()

,language,repo,readme,readme_clean,readme_stemmed
0,java,/roboguice/roboguice,"As of August 2016, RoboGuice is no longer supp...",august 2016 roboguice longer supported nearly ...,august 2016 roboguic longer support nearli 5 y...
1,java,/kikoso/android-stackblur,Android StackBlur\nAndroid StackBlur is a libr...,android stackblur android stackblur library pe...,android stackblur android stackblur librari pe...
2,java,/avast/android-butterknife-zelezny,ButterKnifeZelezny\nSimple plug-in for Android...,butterknifezelezny simple plugin android studi...,butterknifezelezni simpl plugin android studio...
3,java,/bauerca/drag-sort-listview,DragSortListView\nNOTICE: No longer maintained...,dragsortlistview notice longer maintained much...,dragsortlistview notic longer maintain much ti...
4,java,/airbnb/native-navigation,Native Navigation\n\n\n\n\nDISCLAIMER\nThis pr...,native navigation disclaimer project currently...,nativ navig disclaim project current beta mani...


In [212]:
df['readme_lem'] = df.readme_clean.apply(lemmatize)

In [213]:
df.head()

,language,repo,readme,readme_clean,readme_stemmed,readme_lem
0,java,/roboguice/roboguice,"As of August 2016, RoboGuice is no longer supp...",august 2016 roboguice longer supported nearly ...,august 2016 roboguic longer support nearli 5 y...,august 2016 roboguice longer supported nearly ...
1,java,/kikoso/android-stackblur,Android StackBlur\nAndroid StackBlur is a libr...,android stackblur android stackblur library pe...,android stackblur android stackblur librari pe...,android stackblur android stackblur library pe...
2,java,/avast/android-butterknife-zelezny,ButterKnifeZelezny\nSimple plug-in for Android...,butterknifezelezny simple plugin android studi...,butterknifezelezni simpl plugin android studio...,butterknifezelezny simple plugin android studi...
3,java,/bauerca/drag-sort-listview,DragSortListView\nNOTICE: No longer maintained...,dragsortlistview notice longer maintained much...,dragsortlistview notic longer maintain much ti...,dragsortlistview notice longer maintained much...
4,java,/airbnb/native-navigation,Native Navigation\n\n\n\n\nDISCLAIMER\nThis pr...,native navigation disclaimer project currently...,nativ navig disclaim project current beta mani...,native navigation disclaimer project currently...


## Explore